In this notebook, we will show how to use BTMFSI for imputations and predictions.

In [1]:
import pandas as pd
import numpy as np
from numpy.random import multivariate_normal as mvnrnd
from scipy.stats import wishart
from scipy.stats import invwishart
from numpy.linalg import inv as inv
import random
import btmfsi as bs
import seaborn as sns
import matplotlib.pyplot as  plt

mm_scale scales a matrix between 0 and 1.

In [2]:
def mm_scale(dense_mat, train_size):
    training_data=dense_mat[:,:train_size]
    train_min = np.min(training_data[np.nonzero(training_data)])
    epsilon = abs(train_min)*0.01
    val_max = training_data.max()
    val_min = train_min - epsilon
    result_mat = ((dense_mat - val_min)/(val_max - val_min))
    return result_mat, val_max, val_min

mvmat produce a matrix with missing values

In [3]:
def mv_mat(dense_mat, mv_rate):
	lenrow, lencol = dense_mat.shape[0], dense_mat.shape[1]
	binary_mat = np.ones(lenrow*lencol)
	lenmv = int(lenrow*lencol*mv_rate)
	binary_mat[:lenmv] = 0
	np.random.shuffle(binary_mat)
	result_mat = np.multiply(dense_mat, binary_mat.reshape(lenrow, lencol))
	return result_mat, binary_mat.reshape(lenrow, lencol)  

The function FrameSyn and PlotFrame are used to plot the prediction values of time-series gene expressions.

In [4]:
def FrameSyn(np_arr, label, time, data):
    df_syn = pd.DataFrame({'Gene expression': np_arr, 'Method': label, 'Time point': time, 'Data': data})
    return df_syn

def PlotFrame(dense_mat, mat_hat, row_select):
    time_length = mat_hat.shape[1]
    time = np.arange(time_length)
    sample_size = dense_mat.shape[0]-1
    df_list = []
    for i in range(row_select):
        name = 'sample '+ str(i+1)
        j = random.randint(0, sample_size)
        df_ob = FrameSyn(dense_mat[j,-time_length:], 'Observation', time, name)
        df_btmfsi = FrameSyn(mat_hat[j,:], 'BTMFSI', time, name)
        df_temp = pd.concat([df_ob, df_btmfsi], axis=0)
        df_list.append(df_temp)
    df_all = pd.concat(df_list, axis=0)
    return df_all

The following code is used to load the full matrix of time-series gene expression and side information (additional information) in numpy format.

In [5]:
#To load the synthetic data, please use the following code instead:
#----
dense_mat_ = np.load('./data/synthetic_data.npy')
additional_info = np.load('./data/synthetic_data_addinfo.npy')
#----

#To load the yeast cell cycle data, please use the following code:
#----
#dense_mat = np.load('./data/ycc800_data.npy')
#additional_info = np.load('./data/ycc800_data_addinfo.npy')
#----

#dense_mat = np.load('./data/ymc800_data_origin.npy')
#dense_mat = np.log10(dense_mat)
#additional_info = np.load('./data/ymc800_data_addinfo.npy')

We use mm_scale function to normalize the multiple time-series gene expression data. 
The miss_rate represents the ratio of missing value in our paper. 
The term sparse_mat represents the time-series gene expression data with missing values, which can be obtained using function mvmat.

In [6]:
train_size= 20
miss_rate = 0.2
sparse_origin, binary_mat = mv_mat(dense_mat_, miss_rate)
sparse_mat_, val_max, val_min = mm_scale(sparse_origin, train_size)
sparse_mat = np.multiply(sparse_mat_, binary_mat)
dense_mat = (dense_mat_-val_min)/(val_max-val_min)
val = abs(val_max - val_min)

Please check prefunc.py for the meaning of the following parameters.

In [7]:
alpha = 1
pred_time_steps = 20 * 1
multi_steps = 1
rank = 2
time_lags = np.array([1, 2, 3, 4, 5, 6])
maxiter = np.array([200, 100, 20, 10])

In [ ]:
mat_prediction, imputation_rmse, prediction_rmse  = bs.multi_prediction_side(dense_mat, sparse_mat, additional_info, alpha, pred_time_steps, multi_steps, rank, time_lags, maxiter)

In [ ]:
#normalized imputation rmse
imputation_rmse

In [ ]:
#original imputation rmse
imputation_rmse*val

In [ ]:
#normalized prediction rmse
prediction_rmse

In [ ]:
#original prediction rmse
prediction_rmse*val

The term 'num=5' means that 5 randomly selected predictions will be listed below.

In [ ]:
num = 5
df_plot = PlotFrame(dense_mat, mat_prediction, num)
g = sns.FacetGrid(df_plot, col="Data", hue="Method", 
    palette='bright', margin_titles=True)
g.map(sns.lineplot, "Time point", "Gene expression").add_legend()